# Modeling with GNN-ePC-SAFT

Model combining graph neural network with ePC-SAFT


## Starting point

In [1]:
%pip install torchmetrics clu ml-collections torch_geometric polars rdkit -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradient 2.0.6 requires attrs<=19, but you have attrs 23.1.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -U "ray[tune]" -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.23.4 which is incompatible.
tensorboard 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.23.4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install torch_scatter torch_sparse -f https://data.pyg.org/whl/torch-1.12.1+cu116.html -q

In [3]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

1.12.1+cu116


In [4]:
%pip show jax | grep Version

Version: 0.4.1
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import wandb
wandb.login()

In [5]:
!nvidia-smi

Sun Jul 23 17:22:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro P5000        Off  | 00000000:00:05.0 Off |                  Off |
| 26%   35C    P8     6W / 180W |      2MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## PNA EPCSAFT

In [ ]:
!python train.py --workdir=./ --config=configs/default.py \
--config.num_train_steps={int(500000)} --config.log_every_steps=100 --config.num_para=7\
--config.checkpoint_every_steps=2000 --config.learning_rate=0.0001 --config.warmup_steps=100 --config.optimizer={'adam'} --config.batch_size=128\
--config.propagation_depth=5 --config.hidden_dim=512 --config.num_mlp_layers=2 --config.pre_layers=2 --config.post_layers=1

In [ ]:
!python tuner_restore.py --workdir=/notebooks/GNN-ePC-SAFT/ --config=configs/default.py \
--restoredir=./ray/tune_with_parameters_2023-07-24_02-37-22 \
--config.num_train_steps=6000 --config.log_every_steps=100\
--config.learning_rate=0.001 --config.warmup_steps=100 --config.optimizer={'adam'} --config.batch_size=128\
--config.num_para=7

x64 on
2023-07-24 02:53:43,950	ERROR utils.py:540 -- Unexpected error calculating docker cpuset ids.
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/ray/_private/utils.py", line 537, in _get_docker_cpus
    cpu_ids.append(int(num_or_range))
ValueError: invalid literal for int() with base 10: '\n'
2023-07-24 02:53:43,998	INFO worker.py:1621 -- Started a local Ray instance.
2023-07-24 02:53:45,208	INFO experiment_analysis.py:972 -- No trial data passed in during `ExperimentAnalysis` initialization -- you are most likely loading the experiment after it has completed.
Loading trial data from the experiment checkpoint file. This may result in loading some stale information, since checkpointing is periodic.
2023-07-24 02:53:45,221	WARNING experiment_analysis.py:916 -- Failed to read the results for 16 trials:
- ray/tune_with_parameters_2023-07-24_02-37-22/tune_with_parameters_04508_00003_3_hidden_dim=128,num_mlp_layers=1,post_layers=1,pre_layers=2,propagatio

In [ ]:
!python evaluate.py --workdir=./ --config=configs/default.py \
--config.num_train_steps={int(500000)} --config.log_every_steps=100 --config.num_para=7\
--config.checkpoint_every_steps=2000 --config.learning_rate=0.00001 --config.warmup_steps=100 --config.optimizer={'adam'} --config.batch_size=128\
--config.propagation_depth=5 --config.hidden_dim=512 --config.num_mlp_layers=2 --config.pre_layers=2 --config.post_layers=1